In [237]:
import time
import numba
import numpy as np
from numba import guvectorize, float32, float64

In [238]:
# The target keyword decorator for loss and error return

def gu_activate(func, *args, **kwargs):
    """
    The guvectorize decorated method, runs in nopython mode
    :param func: a loss error method
    :param target: choose between | 1: None -> serial | 2: 'parallel' -> parallel | execution method
    :return: the decorated loss, error function
    """
    kwargs_ = {k: v for k, v in kwargs.items() if v is not None}
    return guvectorize([(float32[:], float32[:]),
                      (float64[:], float64[:])],
                     '(n)->(n)', nopython=True, fastmath=True, *args, **kwargs_)(func)

In [239]:
def relu(x, y):
    for i in range(x.shape[0]):
        y[i] = max(0, x[i])

def relu_d(x, y):
    for i in range(x.shape[0]):
        y[i] = 1 if x[i] > 0 else 0

def tanh(x, y):
    for i in range(x.shape[0]):
        y[i] = np.tanh(x[i])

def tanh_d(x, y):
    for i in range(x.shape[0]):
        y[i] = 1 - x[i]**2

def sigmoid(x, y):
    for i in range(x.shape[0]):
        y[i] = 1./(1 + np.exp(-x[i]))

def sigmoid_d(x, y):
    for i in range(x.shape[0]):
        y[i] = x[i] - x[i]**2

def py_relu(x):
    x[x<0] = 0
    return x
def py_d_relu(y):
    y[y>0] = 1
    return y
def py_tanh(x):
    return np.tanh(x)
def py_d_tanh(y):
    return 1 - y**2

def py_sigmoid(x):
    return 1./(1 + np.exp(-x))
def py_d_sigmoid(y):
    return y - y**2

In [240]:
neurons = np.random.uniform(-1, 1, 50000000).astype(np.float32)
neurons

array([-0.9001456 , -0.02389985,  0.63318   , ..., -0.5215977 ,
       -0.07095645,  0.3941204 ], dtype=float32)

In [241]:
s_relu = gu_activate(relu)
p_relu = gu_activate(relu, target='parallel')
s_relu_d = gu_activate(relu_d)
p_relu_d = gu_activate(relu_d, target='parallel')

s_tanh = gu_activate(tanh)
p_tanh = gu_activate(tanh, target='parallel')
s_tanh_d = gu_activate(tanh_d)
p_tanh_d = gu_activate(tanh_d, target='parallel')

s_sigmoid = gu_activate(sigmoid)
p_sigmoid = gu_activate(sigmoid, target='parallel')
s_sigmoid_d = gu_activate(sigmoid_d)
p_sigmoid_d = gu_activate(sigmoid_d, target='parallel')

In [242]:
def ntime(func1):
    start = time.time()
    func1(neurons)
    end = time.time()
    print("Elapsed Numba Pre = %s" % (end - start))
    start = time.time()
    f2 = func1(neurons)
    end = time.time()
    print("Elapsed Numba Post = %s" % (end - start))
    return f2

def ptime(func1):
    start = time.time()
    f1 = func1(neurons)
    end = time.time()
    print("Elapsed PyTime = %s" % (end - start))
    return f1

In [243]:
ntime(s_sigmoid), ptime(py_sigmoid)

Elapsed Numba Pre = 0.28623390197753906
Elapsed Numba Post = 0.2792844772338867
Elapsed PyTime = 0.34105682373046875


(array([0.2890206 , 0.49402535, 0.65321016, ..., 0.37247872, 0.48226833,
        0.5972742 ], dtype=float32),
 array([0.2890206 , 0.49402538, 0.65321016, ..., 0.37247872, 0.48226833,
        0.5972742 ], dtype=float32))